---

## 2024-1학기 인터랙티브 그래프마이닝

# ✅ COVID-19 가짜 뉴스 탐지 (Guide Code)

---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd '/content/drive/MyDrive/graph'

/content/drive/MyDrive/graph


In [ ]:
import pandas as pd
import numpy as np

# 1. Data Description
- id : claim id
- label : 0(normal) / 1(fake)
- published_date : 보도 날짜
- keybert_keywords : kebert를 통해 추출한 기사 키워드
- ner_keywords : ner를 통해 추출한 기사 키워드
- youtube0 ~ youtube9 : 해당 키워드로 유트브에 검색했을 때 나오는 상위 10개의 유투브의 제목, 설명 텍스트

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [ ]:
# train data: 5277개의 기사로 구성
train

,id,label,claim,published_date,keybert_keywords,ner_keywords,youtube0,youtube1,youtube2,youtube3,youtube4,youtube5,youtube6,youtube7,youtube8,youtube9
0,0,0,Did a Vermont Woman Post a Joke About Trump Br...,2018-01-25,"['trump', 'vermont', 'woman']","['vermont', 'trump', 'woman']",Citizen Trump A talk given at the University ...,News Donald Trump has a great woman problem N...,Trump Supporters Remain Despite Vulgar Slur J...,BREAKING Donald trump has a great woman proble...,Revolutionary War SNL The Patriots from New E...,World News Donald Trump has a great woman prob...,Latest News Donald Trump has a great woman pro...,AM Executive Session Gender Matters The U S Pr...,Trump Discusses Tax Bill Gives ;Total Support ...,Chicago s Women s March January This is what ...
1,1,1,Were Baseball Players Photographed Kneeling to...,2017-11-08,"['lynchings', 'baseball', 's']","['lynchings', 'baseball', 's']",Tony Shalhoub Still ;Monk ; after all these ye...,Lesson Class differences and racial variation ...,Mass Media Mass Culture and the Golden Age of ...,Mark W Bennett quot;Implicit Bias and the Law ...,Chris Hedges Writing as Resistance presented b...,Race in America FOCIS th Anniversary lecture s...,,,,
2,2,0,Wisconsin state Rep. John Nygren hits pay dirt...,2019-02-07,"['tax', 'wisconsin', 'dirt']","['rep', 'wisconsin', 'tax', 'wisconsin']",Pocan Shutdown Waste of Time Energy Pocan Shu...,Lawmaker This couldn ;t be more embarrassing f...,State Rep Bob Behning Creating teacher career ...,US GOV th,Representative John Macco of the th talks the ...,Democratic Senator Calls On Va Gov To Resign M...,Rep Sean Duffy introduces the Reciprocal Trade...,Rep Dan Caulkins on IL Minimum Wage Hike Stat...,Rep Weber Worried More Jobs Will Leave IL Rea...,Foxconn May Not Build B Wisconsin Plant Presid...
3,3,0,Progressive group accuses Senate splinter grou...,2017-11-04,"['donors', 'cashing', 'senate']","['cashing', 'senate', 'donors']",Roy Moore s Attorney Refers To News Anchor ;s ...,Documents that Changed the Way We Live Docume...,Time to Wake Up EPA Nominees Show Blatant Disr...,Училище по персонализирана медицина Панел Учи...,Conference Theme Panel Economics for a New Pro...,,,,,
4,4,0,Seal with Unusual Stripe Pattern Markings Spot...,2016-10-21,"['seal', 'washington', 'markings']","['seal', 'washington', 'markings']",Parking Lot Striping Garland Texas Asphalt st...,Learn about the MAHLE Original Gasket Line Bi...,Austroads Guide to Road Design Part Session of,I Major Deck and Superstructure Rehabilitation...,The Premier Parking Lot Striping Company in Au...,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5272,5272,0,Does Hillary Clinton want a $15 or $12 minimum...,2016-04-14,"['hillary', 'clinton']","['hillary clinton', 'hillary', 'clinton']",Hillary Clinton Addresses Her Losing Streak Co...,Stronger Together Hillary Clinton Donald Trum...,Love and Kindness Hillary Clinton Hillary Cli...,Hillary Clinton Interview at The Breakfast Clu...,Hillary Clinton Reveals Bill ;s Most Annoying ...,Stephen Interviews Hillary Clinton At Carnegie...,Hillary Clinton ;s entire Pennsylvania primary...,Hillary Clinton on NY Primary Win FULL SPEECH ...,Thank you New York Hillary Clinton In her New...,Hillary Clinton ;s full New York primary victo...
5273,5273,1,George Soros s son isn t married to Adam Schif...,2019-10-10,"['sister', 'married', 'son']","['sister', 'married', 'son']",Norman Lusts After His Mother Bates Motel Scen...,Cenk ve Cemre Evlendi Şeniz Çıldırdı Zalim İs...,Seven Year Old Mykal Michelle Harris on Mixed ...,My Sister Song CoComelon Nursery Rhymes amp; K...,quot;He annoys me so much quot; Naomi Osaka On...,MA SŒUR SE MARIE JUST MARRIED Lévanah amp;Fami...,Sheikh Mohammed bin Rashid ;s daugher Maryam m...,Amelia and Link Tell Meredith About the Baby G...,Tamron Hall Talks with Her Nephew About the Mu...,Thomas Doherty Juliet Doherty amp; Harry Jarvi...
5274,5274,0,Sanders: It's not impossible to get enough del...,2016-

In [ ]:
# test data: 1856개의 기사로 구성 (test에는 라벨값이 없으므로, 이를 예측하여 submission 해야 함.)
test

,id,claim,published_date,keybert_keywords,ner_keywords,youtube0,youtube1,youtube2,youtube3,youtube4,youtube5,youtube6,youtube7,youtube8,youtube9
0,0,According to the CDC so far this year Florida ...,2020-05-27,['pneumonia'],"['florida', 'pneumonia', 'cdc']",Watch Full Coronavirus Coverage May NBC News N...,Watch Full Coronavirus Coverage May NBC News N...,Ask the expert Here ;s why screening is a good...,MORE Questions for an Infectious Disease Speci...,Coronavirus and Cuba,Understanding and Managing Coronavirus in SNF ...,SARS CoV Pandemic Caltech Seminar Day Online S...,CAH Swingbed Initiative with Kerry Dunning Ke...,Diabetics and Novel Coronavirus ▻ Theresa De ...,CDPHE explains how it classifies COVID deaths ...
1,1,Claim that Rep. Alexandria Ocasio-Cortez tweet...,2020-06-23,"['november', 'elections', 'tweeted']","['covid', 'us', 'rep']",Maps show explosive growth of coronavirus in U...,Rep Raja Krishnamoorthi on Continued Fallout o...,Biologist says young people are fueling Covid ...,Former Allergan CEO on the most likely timelin...,Rep DeGette questions top U S health officials...,Rep Kevin McCarthy on Covid spikes reopening t...,Rep Kevin Brady on back to work bonus proposal...,Florida and Texas struggle with surging corona...,What flying in the U S amid the coronavirus pa...,U S Virus Surge Ample Opportunity for Covid to...
2,2,Models projecting COVID-19 deaths are talking ...,2020-05-05,"['deaths', 'mitigation']","['deaths', 'covid', 'mitigation']",President Donald Trump downplays models projec...,Trump keeps predicting coronavirus death tolls...,Trump Claims New Estimate Predicting More Coro...,Is COVID Seasonal View CME information and cl...,The Bubonic Plague and COVID – Two Diseases in...,Coronavirus outbreak B C reports new cases add...,Coronavirus Number of UK deaths decline from p...,Coronavirus model projects deaths in U S to ne...,Modeling COVID Transmission and Containment in...,UK records most Covid deaths in Europe but Raa...
3,3,A video has been viewed tens of thousands of t...,2020-05-02,"['instagram', 'facebook', 'tea']","['covid', 'uk', 'boris johnson']",Coronavirus Boris Johnson updates nation on Co...,Coronavirus How Boris Johnson government ;s po...,Boris Johnson unveils new COVID lockdown rules...,Coronavirus Public challenge Boris Johnson ;s ...,Coronavirus UK now ;past the peak ; says Boris...,UK PM Boris Johnson returns to work after reco...,Boris Johnson We are past the peak of disease ...,In full Boris Johnson says he won ;t lift UK c...,Boris Johnson s new stay alert message splits ...,Watch Back Boris Johnson Addresses UK On State...
4,4,A video shows how Indian police frightened tho...,2020-04-24,"['ambulance', 'police', 'frightened']","['covid', 'indian', 'ambulance']",Watch TN Police put lockdown violators in ambu...,Fake Covid patient TN Police releases awarenes...,TN Police ;Prank ; Lockdown Violators With Fak...,Police drone catches teens violating COVID loc...,India s mobile coronavirus testing van hits Ne...,Ambulance Traffic Jams At Moscow Hospitals As ...,;India ;s Covid recovery rate ; Union govt on ...,Police drone interrupts couple ;s romantic get...,Watch Tamil Nadu Police put lockdown violators...,Pak Reaction on Indian Police Prank Offenders ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851,1851,COMMENTARY: COVID-19 Diary Day 6: Clinicians' ...,2020-04-10,"['clinicians', 'diary']","['clinicians', 'diary']",Adding Services and Products,Adding Clinicians,Acting Director ;s Message April,Adding and Editing Account Users,Advice for New Therapists Subscribe now for m...,COVID Clinical Webinar,Diary of a Researcher Day April Libby Researc...,THE HARDEST SURGERY OPERATION I ;VE EVER EXPER...,Configuring Business Basics,Quick Navigational Tips
1852,1852,Facebook and Instagram posts shared thousands ...,2020-05-25,"['instagram', 'facebook', 'vaccines']","['instagram', 'facebook', 'vaccines']",Ousted vaccine chief testifies Americans deser...,Anti Vaxxers Could

In [ ]:
# test 데이터 내 1856개의 노드에 대해 예측된 라벨 (sample에서는 모두 0으로 초기화되어 있음.)
submission

,ID,Label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
1851,1851,0
1852,1852,0
1853,1853,0
1854,1854,0


# 2. Graph Example

In [ ]:
!pip install torch==2.2.1+cu121 torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.3/757.3 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 975.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
import torch
print("PyTorch has version {}".format(torch.__version__))
# install dgl
# https://www.dgl.ai/pages/start.html

PyTorch has version 2.2.1+cu121


In [ ]:
# Install DGL library
!pip install  dgl -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/

In [ ]:
import numpy as np
from scipy.sparse import coo_matrix
import torch
import dgl

In [ ]:
# adjacency matrix 구축 예시: 같은 월에 작성된 claim끼리 연결
train['published_date'] = pd.to_datetime(train['published_date'], format='%Y-%m-%d', errors='raise')
train

,id,label,claim,published_date,keybert_keywords,ner_keywords,youtube0,youtube1,youtube2,youtube3,youtube4,youtube5,youtube6,youtube7,youtube8,youtube9
0,0,0,Did a Vermont Woman Post a Joke About Trump Br...,2018-01-25,"['trump', 'vermont', 'woman']","['vermont', 'trump', 'woman']",Citizen Trump A talk given at the University ...,News Donald Trump has a great woman problem N...,Trump Supporters Remain Despite Vulgar Slur J...,BREAKING Donald trump has a great woman proble...,Revolutionary War SNL The Patriots from New E...,World News Donald Trump has a great woman prob...,Latest News Donald Trump has a great woman pro...,AM Executive Session Gender Matters The U S Pr...,Trump Discusses Tax Bill Gives ;Total Support ...,Chicago s Women s March January This is what ...
1,1,1,Were Baseball Players Photographed Kneeling to...,2017-11-08,"['lynchings', 'baseball', 's']","['lynchings', 'baseball', 's']",Tony Shalhoub Still ;Monk ; after all these ye...,Lesson Class differences and racial variation ...,Mass Media Mass Culture and the Golden Age of ...,Mark W Bennett quot;Implicit Bias and the Law ...,Chris Hedges Writing as Resistance presented b...,Race in America FOCIS th Anniversary lecture s...,,,,
2,2,0,Wisconsin state Rep. John Nygren hits pay dirt...,2019-02-07,"['tax', 'wisconsin', 'dirt']","['rep', 'wisconsin', 'tax', 'wisconsin']",Pocan Shutdown Waste of Time Energy Pocan Shu...,Lawmaker This couldn ;t be more embarrassing f...,State Rep Bob Behning Creating teacher career ...,US GOV th,Representative John Macco of the th talks the ...,Democratic Senator Calls On Va Gov To Resign M...,Rep Sean Duffy introduces the Reciprocal Trade...,Rep Dan Caulkins on IL Minimum Wage Hike Stat...,Rep Weber Worried More Jobs Will Leave IL Rea...,Foxconn May Not Build B Wisconsin Plant Presid...
3,3,0,Progressive group accuses Senate splinter grou...,2017-11-04,"['donors', 'cashing', 'senate']","['cashing', 'senate', 'donors']",Roy Moore s Attorney Refers To News Anchor ;s ...,Documents that Changed the Way We Live Docume...,Time to Wake Up EPA Nominees Show Blatant Disr...,Училище по персонализирана медицина Панел Учи...,Conference Theme Panel Economics for a New Pro...,,,,,
4,4,0,Seal with Unusual Stripe Pattern Markings Spot...,2016-10-21,"['seal', 'washington', 'markings']","['seal', 'washington', 'markings']",Parking Lot Striping Garland Texas Asphalt st...,Learn about the MAHLE Original Gasket Line Bi...,Austroads Guide to Road Design Part Session of,I Major Deck and Superstructure Rehabilitation...,The Premier Parking Lot Striping Company in Au...,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5272,5272,0,Does Hillary Clinton want a $15 or $12 minimum...,2016-04-14,"['hillary', 'clinton']","['hillary clinton', 'hillary', 'clinton']",Hillary Clinton Addresses Her Losing Streak Co...,Stronger Together Hillary Clinton Donald Trum...,Love and Kindness Hillary Clinton Hillary Cli...,Hillary Clinton Interview at The Breakfast Clu...,Hillary Clinton Reveals Bill ;s Most Annoying ...,Stephen Interviews Hillary Clinton At Carnegie...,Hillary Clinton ;s entire Pennsylvania primary...,Hillary Clinton on NY Primary Win FULL SPEECH ...,Thank you New York Hillary Clinton In her New...,Hillary Clinton ;s full New York primary victo...
5273,5273,1,George Soros s son isn t married to Adam Schif...,2019-10-10,"['sister', 'married', 'son']","['sister', 'married', 'son']",Norman Lusts After His Mother Bates Motel Scen...,Cenk ve Cemre Evlendi Şeniz Çıldırdı Zalim İs...,Seven Year Old Mykal Michelle Harris on Mixed ...,My Sister Song CoComelon Nursery Rhymes amp; K...,quot;He annoys me so much quot; Naomi Osaka On...,MA SŒUR SE MARIE JUST MARRIED Lévanah amp;Fami...,Sheikh Mohammed bin Rashid ;s daugher Maryam m...,Amelia and Link Tell Meredith About the Baby G...,Tamron Hall Talks with Her Nephew About the Mu...,Thomas Doherty Juliet Doherty amp; Harry Jarvi...
5274,5274,0,Sanders: It's not impossible to get enough del...,2016-

In [ ]:
# adjacency matrix 초기화
dic_array = np.zeros((len(train),len(train)))
print(dic_array.shape)
print(dic_array.sum())

(5277, 5277)
0.0


In [ ]:
# 원하는 연결 구조에 따라 adjacency matrix에 값 추가
for i, (i_claim, i_m) in enumerate(zip(train.claim, train.published_date.dt.month)):
  for j, (j_claim, j_m) in enumerate(zip(train.claim, train.published_date.dt.month)):
    if i_m == j_m:
      dic_array[i][j] += 1

In [ ]:
# 완성된 Adjacency matrix 및 엣지 합
print(dic_array.shape)
print(dic_array.sum())

(5277, 5277)
2758845.0


In [ ]:
# feature 구축 예시 : random하게 node 당 dim 10의 feature 생성
feature = np.random.rand(dic_array.shape[0], 10).astype(np.float32)  # 데이터 유형을 float32로 설정

In [ ]:
# build graph 예시 : homogeneous graph

# build graph
edge_index = torch.nonzero(torch.tensor(dic_array), as_tuple=False).T
g = dgl.graph((edge_index[1], edge_index[0]))

# assign features
g.ndata['feature'] = torch.tensor(feature, dtype=torch.float32)
g.edata['weight'] = torch.tensor(coo_matrix(dic_array).data, dtype=torch.float32)

In [ ]:
coo_matrix(torch.tensor(dic_array)).data

array([1., 1., 1., ..., 1., 1., 1.])

In [ ]:
g

Graph(num_nodes=5277, num_edges=2758845,
      ndata_schemes={'feature': Scheme(shape=(10,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float32)})

이외에 학습을 위해 필요한 요소를 할당하여 그래프에 저장 후 학습을 수행하세요.
ex) labels, train_mask, valid_mask 등

In [5]:
!pip install torch==2.2.1+cu121 torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.3/757.3 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 4.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 107.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 104.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions o

In [1]:
import torch
print("PyTorch has version {}".format(torch.__version__))
# install dgl
# https://www.dgl.ai/pages/start.html

PyTorch has version 2.2.1+cu121


In [2]:
# Install DGL library
!pip install  dgl -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 98.7 MB/s eta 0:00:00


In [3]:
# 필요한 라이브러리 설치
!pip install torch torch_geometric

In [ ]:
import pandas as pd
import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch
import torch.nn.functional as F
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 로드
path = '/content/drive/MyDrive/graph/'
train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')
submission_df = pd.read_csv(path + 'sample_submission.csv')

# 데이터 전처리
def preprocess(df):
    keywords = df['ner_keywords'].fillna('') + ' ' + df['keybert_keywords'].fillna('')
    keywords = keywords.str.split()
    return keywords

train_keywords = preprocess(train_df)
test_keywords = preprocess(test_df)

# 노드와 엣지 생성
vectorizer = TfidfVectorizer(max_features=1000)
train_tfidf = vectorizer.fit_transform(train_keywords.apply(lambda x: ' '.join(x)))
test_tfidf = vectorizer.transform(test_keywords.apply(lambda x: ' '.join(x)))

train_similarity = cosine_similarity(train_tfidf)
test_similarity = cosine_similarity(test_tfidf, train_tfidf)

# 노드 feature와 엣지 인덱스 생성
def create_graph_data(similarity, labels=None):
    edge_index = np.array(similarity.nonzero(), dtype=np.int64)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    x = torch.tensor(similarity, dtype=torch.float)
    if labels is not None:
        y = torch.tensor(labels, dtype=torch.long)
        return Data(x=x, edge_index=edge_index, y=y)
    return Data(x=x, edge_index=edge_index)

train_data = create_graph_data(train_tfidf.toarray(), train_df['label'].values)
test_data = create_graph_data(test_tfidf.toarray())

# GNN 모델 정의
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        return x

# 하이퍼파라미터 설정
input_dim = train_tfidf.shape[1]  # 입력 차원을 TF-IDF 특성 수로 설정
hidden_dim = 128
output_dim = 2
learning_rate = 0.001
epochs = 200

# 모델 초기화 및 옵티마이저 설정
model = GCN(input_dim, hidden_dim, output_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 조기 종료를 위한 변수 설정
early_stopping_patience = 20
best_loss = float('inf')
early_stopping_counter = 0

# 모델 학습
model.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    out = model(train_data)

    # out의 크기와 train_data.y의 크기 확인
    if out.size(0) != train_data.y.size(0):
        print(f'Size mismatch: model output size {out.size(0)}, target size {train_data.y.size(0)}')
        break

    loss = F.cross_entropy(out, train_data.y)
    loss.backward()
    optimizer.step()

    # 조기 종료 로직
    if loss < best_loss:
        best_loss = loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= early_stopping_patience:
        print(f'Early stopping at epoch {epoch}')
        break

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

    # 각 에포크마다 테스트 데이터에 대한 정확도 출력
    model.eval()
    with torch.no_grad():
        test_out = model(test_data)
        pred = test_out.argmax(dim=1).numpy()
        # 정확도 계산 대신 예측값을 출력합니다.
        print(f'Epoch {epoch}, Predictions: {pred}')

# 예측 수행
model.eval()
with torch.no_grad():
    test_out = model(test_data)
    pred = test_out.argmax(dim=1).numpy()

# 제출 파일 생성
submission_df['Label'] = pred
submission_df.to_csv(path + 'submission_1e-1.csv', index=False)
print('Submission file saved!')

Epoch 0, Loss: 0.6938698887825012
Epoch 0, Predictions: [0 1 0 ... 0 0 1]
Epoch 1, Predictions: [0 0 0 ... 0 0 0]
Epoch 2, Predictions: [0 0 0 ... 0 0 0]
Epoch 3, Predictions: [0 0 0 ... 0 0 0]
Epoch 4, Predictions: [0 0 0 ... 0 0 0]
Epoch 5, Predictions: [0 0 0 ... 0 1 0]
Epoch 6, Predictions: [0 0 0 ... 0 1 0]
Epoch 7, Predictions: [0 0 0 ... 0 1 0]
Epoch 8, Predictions: [0 0 0 ... 0 1 0]
Epoch 9, Predictions: [0 0 0 ... 0 1 0]
Epoch 10, Loss: 0.6696652173995972
Epoch 10, Predictions: [0 0 0 ... 0 1 0]
Epoch 11, Predictions: [0 0 0 ... 0 1 0]
Epoch 12, Predictions: [0 0 0 ... 0 1 0]
Epoch 13, Predictions: [0 0 0 ... 0 1 0]
Epoch 14, Predictions: [0 0 0 ... 0 1 0]
Epoch 15, Predictions: [0 0 0 ... 0 1 0]
Epoch 16, Predictions: [0 0 0 ... 0 1 0]
Epoch 17, Predictions: [0 0 0 ... 0 1 0]
Epoch 18, Predictions: [0 0 0 ... 0 1 0]
Epoch 19, Predictions: [0 0 0 ... 0 1 0]
Epoch 20, Loss: 0.6162769794464111
Epoch 20, Predictions: [0 0 0 ... 0 1 0]
Epoch 21, Predictions: [0 0 0 ... 0 1 0]
Epoc

In [ ]:
import pandas as pd
import numpy as np
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
import torch
import torch.nn.functional as F
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# 데이터 로드
path = '/content/drive/MyDrive/graph/'
train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')
submission_df = pd.read_csv(path + 'sample_submission.csv')

# 데이터 전처리
def preprocess(df):
    keywords = df['ner_keywords'].fillna('') + ' ' + df['keybert_keywords'].fillna('')
    keywords = keywords.str.split()
    return keywords

train_keywords = preprocess(train_df)
test_keywords = preprocess(test_df)

# 노드와 엣지 생성
vectorizer = TfidfVectorizer(max_features=1000)
train_tfidf = vectorizer.fit_transform(train_keywords.apply(lambda x: ' '.join(x)))
test_tfidf = vectorizer.transform(test_keywords.apply(lambda x: ' '.join(x)))

train_similarity = cosine_similarity(train_tfidf)
test_similarity = cosine_similarity(test_tfidf, train_tfidf)

# 노드 feature와 엣지 인덱스 생성
def create_graph_data(similarity, labels=None):
    edge_index = np.array(similarity.nonzero(), dtype=np.int64)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    x = torch.tensor(similarity, dtype=torch.float)
    if labels is not None:
        y = torch.tensor(labels, dtype=torch.long)
        return Data(x=x, edge_index=edge_index, y=y)
    return Data(x=x, edge_index=edge_index)

train_data = create_graph_data(train_tfidf.toarray(), train_df['label'].values)
test_data = create_graph_data(test_tfidf.toarray())

# GNN 모델 정의
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        return x

# 하이퍼파라미터 설정
input_dim = train_tfidf.shape[1]  # 입력 차원을 TF-IDF 특성 수로 설정
hidden_dim = 128
output_dim = 2
learning_rate = 0.001
epochs = 200
batch_size = 32

# 모델 초기화 및 옵티마이저 설정
model = GCN(input_dim, hidden_dim, output_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 조기 종료를 위한 변수 설정
early_stopping_patience = 20
best_loss = float('inf')
early_stopping_counter = 0

# 모델 학습
model.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    out = model(train_data)

    # out의 크기와 train_data.y의 크기 확인
    if out.size(0) != train_data.y.size(0):
        print(f'Size mismatch: model output size {out.size(0)}, target size {train_data.y.size(0)}')
        break

    loss = F.cross_entropy(out, train_data.y)
    loss.backward()
    optimizer.step()

    # 조기 종료 로직
    if loss < best_loss:
        best_loss = loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= early_stopping_patience:
        print(f'Early stopping at epoch {epoch}')
        break

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

    # 각 에포크마다 테스트 데이터에 대한 정확도 출력
    model.eval()
    with torch.no_grad():
        test_out = model(test_data)
        pred = test_out.argmax(dim=1).numpy()
        # 정확도 계산 대신 예측값을 출력합니다.
        print(f'Epoch {epoch}, Predictions: {pred}')

# 예측 수행
model.eval()
with torch.no_grad():
    test_out = model(test_data)
    pred = test_out.argmax(dim=1).numpy()

# 제출 파일 생성
submission_df['Label'] = pred
submission_df.to_csv(path + 'submission_1e-1.csv', index=False)
print('Submission file saved!')

Epoch 0, Loss: 0.6930699348449707
Epoch 0, Predictions: [0 1 0 ... 1 1 0]
Epoch 1, Predictions: [0 1 0 ... 0 1 0]
Epoch 2, Predictions: [0 0 0 ... 0 1 0]
Epoch 3, Predictions: [0 1 0 ... 0 1 0]
Epoch 4, Predictions: [0 1 0 ... 0 1 0]
Epoch 5, Predictions: [0 1 0 ... 0 1 0]
Epoch 6, Predictions: [0 1 0 ... 0 1 0]
Epoch 7, Predictions: [0 1 0 ... 0 1 0]
Epoch 8, Predictions: [0 1 0 ... 0 1 0]
Epoch 9, Predictions: [0 1 0 ... 0 1 0]
Epoch 10, Loss: 0.665515661239624
Epoch 10, Predictions: [0 1 0 ... 0 1 0]
Epoch 11, Predictions: [0 1 0 ... 0 1 0]
Epoch 12, Predictions: [0 1 0 ... 0 1 0]
Epoch 13, Predictions: [0 1 0 ... 0 1 0]
Epoch 14, Predictions: [0 1 0 ... 0 1 0]
Epoch 15, Predictions: [0 1 0 ... 0 1 0]
Epoch 16, Predictions: [0 1 0 ... 0 1 0]
Epoch 17, Predictions: [0 1 0 ... 0 1 0]
Epoch 18, Predictions: [0 1 0 ... 0 1 0]
Epoch 19, Predictions: [0 0 0 ... 0 1 0]
Epoch 20, Loss: 0.6078376770019531
Epoch 20, Predictions: [0 0 0 ... 0 1 0]
Epoch 21, Predictions: [0 0 0 ... 0 1 0]
Epoch

In [ ]:
import pandas as pd
import numpy as np
import torch
import dgl
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from dgl.nn.pytorch import GraphConv
from torch.nn import Dropout

# 데이터 로드
train = pd.read_csv('train.csv')

# Claim 텍스트 및 키워드 추출
claims = train['claim'].tolist()
keywords = train['keybert_keywords'].apply(eval).tolist()

# 키워드 간의 관련성 계산
def keyword_similarity(keyword_list):
    keyword_text = [' '.join(kw) for kw in keyword_list]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(keyword_text)
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return similarity_matrix

# 키워드 간의 관련성 계산
keyword_sim = keyword_similarity(keywords)

# 유사도 기반 그래프 생성
threshold = 0.5  # 유사도가 threshold 이상인 경우에만 엣지를 추가
edges = []
for i in range(len(keywords)):
    for j in range(i+1, len(keywords)):
        if keyword_sim[i][j] >= threshold:
            edges.append((i, j))

# 그래프 생성
g = dgl.graph(edges, num_nodes=len(train))

# self-loop 추가
g = dgl.add_self_loop(g)

# label 설정
labels = torch.tensor(train['label'].values, dtype=torch.long)

# Train/validation mask 생성
train_indices, val_indices = train_test_split(np.arange(len(labels)), test_size=0.2, stratify=labels)
train_mask = torch.tensor([i in train_indices for i in range(len(labels))], dtype=torch.bool)
val_mask = torch.tensor([i in val_indices for i in range(len(labels))], dtype=torch.bool)

# 그래프에 label 및 mask 추가
g.ndata['label'] = labels[:g.num_nodes()]
g.ndata['train_mask'] = train_mask
g.ndata['val_mask'] = val_mask

# Claim을 이용한 feature 생성
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(train['claim']).toarray()
features = torch.tensor(tfidf_matrix, dtype=torch.float)

# 그래프에 feature 추가
g.ndata['feat'] = features

# GraphSAGE 모델 정의 (Dropout 추가)
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_feats, h_feats, num_classes, dropout):
        super(GraphSAGE, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, h_feats)
        self.conv3 = GraphConv(h_feats, h_feats)
        self.conv4 = GraphConv(h_feats, num_classes)
        self.dropout = Dropout(dropout)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv2(g, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(g, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(g, h)
        return h

# Hyperparameters
in_feats = features.shape[1]  # feature의 크기 설정
h_feats = 128  # hidden layer 크기
num_classes = len(labels.unique())
learning_rate = 0.001  # 학습률
weight_decay = 5e-4  # weight decay
dropout = 0.5  # dropout rate
num_epochs = 500  # 더 많은 epoch

# 모델 및 옵티마이저 초기화
model = GraphSAGE(in_feats, h_feats, num_classes, dropout)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Training 및 validation 함수 정의
def train(model, g, features, labels, train_mask, optimizer):
    model.train()
    logits = model(g, features)
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

# Early Stopping을 위한 설정
patience = 10
best_val_loss = float('inf')
patience_counter = 0

# Training 및 validation 루프
for epoch in range(num_epochs):
    loss = train(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'], optimizer)
    train_acc = evaluate(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'])
    val_acc = evaluate(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['val_mask'])
    val_loss = F.cross_entropy(model(g, g.ndata['feat'])[val_mask], g.ndata['label'][val_mask]).item()
    print(f"Epoch {epoch:05d} | Loss {loss:.4f} | Train Accuracy {train_acc:.4f} | Val Accuracy {val_acc:.4f} | Val Loss {val_loss:.4f}")

    # Early Stopping 체크
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 00000 | Loss 0.6928 | Train Accuracy 0.5200 | Val Accuracy 0.5123 | Val Loss 0.6927
Epoch 00001 | Loss 0.6934 | Train Accuracy 0.5290 | Val Accuracy 0.5104 | Val Loss 0.6925
Epoch 00002 | Loss 0.6929 | Train Accuracy 0.5402 | Val Accuracy 0.5199 | Val Loss 0.6923
Epoch 00003 | Loss 0.6927 | Train Accuracy 0.5537 | Val Accuracy 0.5322 | Val Loss 0.6920
Epoch 00004 | Loss 0.6920 | Train Accuracy 0.5717 | Val Accuracy 0.5455 | Val Loss 0.6917
Epoch 00005 | Loss 0.6918 | Train Accuracy 0.5890 | Val Accuracy 0.5559 | Val Loss 0.6914
Epoch 00006 | Loss 0.6909 | Train Accuracy 0.6077 | Val Accuracy 0.5739 | Val Loss 0.6910
Epoch 00007 | Loss 0.6910 | Train Accuracy 0.6292 | Val Accuracy 0.5814 | Val Loss 0.6906
Epoch 00008 | Loss 0.6908 | Train Accuracy 0.6446 | Val Accuracy 0.5966 | Val Loss 0.6902
Epoch 00009 | Loss 0.6899 | Train Accuracy 0.6674 | Val Accuracy 0.5994 | Val Loss 0.6896
Epoch 00010 | Loss 0.6891 | Train Accuracy 0.6814 | Val Accuracy 0.6070 | Val Loss 0.6890
Epoch 0001

In [ ]:
import pandas as pd
import torch

# test 데이터 로드
test = pd.read_csv('test.csv')

# Claim 텍스트 및 키워드 추출
test_claims = test['claim'].tolist()
test_keywords = test['keybert_keywords'].apply(eval).tolist()

# Claim을 이용한 feature 생성 (학습 데이터와 동일한 방식)
test_tfidf_matrix = tfidf_vectorizer.transform(test_claims).toarray()
test_features = torch.tensor(test_tfidf_matrix, dtype=torch.float)

# 그래프에 feature 추가
test_g = dgl.graph([], num_nodes=len(test))
test_g = dgl.add_self_loop(test_g)
test_g.ndata['feat'] = test_features

# 모델을 평가 모드로 전환
model.eval()

# 테스트 데이터에 대한 예측 수행
with torch.no_grad():
    test_logits = model(test_g, test_g.ndata['feat'])
    _, test_pred = torch.max(test_logits, dim=1)

# 예측 결과를 DataFrame으로 변환
submission = pd.DataFrame({
    'ID': test['id'],
    'Label': test_pred.numpy()
})

# 예측 결과를 submission.csv 파일로 저장
submission.to_csv('submission_3.csv', index=False)

print("Predictions saved to submission.csv")

Predictions saved to submission.csv


In [10]:
import pandas as pd
import numpy as np
import torch
import dgl
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from dgl.nn.pytorch import GraphConv
from torch.nn import Dropout

# 데이터 로드
train = pd.read_csv('train.csv')

# Claim 텍스트 및 키워드 추출
claims = train['claim'].tolist()
keywords = train['keybert_keywords'].apply(eval).tolist()

# 키워드 간의 관련성 계산
def keyword_similarity(keyword_list):
    keyword_text = [' '.join(kw) for kw in keyword_list]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(keyword_text)
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return similarity_matrix

# 키워드 간의 관련성 계산
keyword_sim = keyword_similarity(keywords)

# 유사도 기반 그래프 생성
threshold = 0.5  # 유사도가 threshold 이상인 경우에만 엣지를 추가
edges = []
for i in range(len(keywords)):
    for j in range(i+1, len(keywords)):
        if keyword_sim[i][j] >= threshold:
            edges.append((i, j))

# 그래프 생성
g = dgl.graph(edges, num_nodes=len(train))

# self-loop 추가
g = dgl.add_self_loop(g)

# label 설정
labels = torch.tensor(train['label'].values, dtype=torch.long)

# Train/validation mask 생성
train_indices, val_indices = train_test_split(np.arange(len(labels)), test_size=0.2, stratify=labels)
train_mask = torch.tensor([i in train_indices for i in range(len(labels))], dtype=torch.bool)
val_mask = torch.tensor([i in val_indices for i in range(len(labels))], dtype=torch.bool)

# 그래프에 label 및 mask 추가
g.ndata['label'] = labels[:g.num_nodes()]
g.ndata['train_mask'] = train_mask
g.ndata['val_mask'] = val_mask

# Claim을 이용한 feature 생성
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(train['claim']).toarray()
features = torch.tensor(tfidf_matrix, dtype=torch.float)

# 그래프에 feature 추가
g.ndata['feat'] = features

# GraphSAGE 모델 정의 (Dropout 추가)
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_feats, h_feats, num_classes, dropout):
        super(GraphSAGE, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, h_feats)
        self.conv3 = GraphConv(h_feats, h_feats)
        self.conv4 = GraphConv(h_feats, h_feats)
        self.conv5 = GraphConv(h_feats, num_classes)
        self.dropout = Dropout(dropout)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv2(g, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(g, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv4(g, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv5(g, h)
        return h

# Hyperparameters
in_feats = features.shape[1]  # feature의 크기 설정
h_feats = 128  # hidden layer 크기
num_classes = len(labels.unique())
learning_rate = 0.0001  # 학습률
weight_decay = 5e-4  # weight decay
dropout = 0.5  # dropout rate
num_epochs = 500  # 더 많은 epoch

# 모델 및 옵티마이저 초기화
model = GraphSAGE(in_feats, h_feats, num_classes, dropout)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Training 및 validation 함수 정의
def train(model, g, features, labels, train_mask, optimizer):
    model.train()
    logits = model(g, features)
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

# Early Stopping을 위한 설정
patience = 10
best_val_loss = float('inf')
patience_counter = 0

# Training 및 validation 루프
for epoch in range(num_epochs):
    loss = train(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'], optimizer)
    train_acc = evaluate(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['train_mask'])
    val_acc = evaluate(model, g, g.ndata['feat'], g.ndata['label'], g.ndata['val_mask'])
    val_loss = F.cross_entropy(model(g, g.ndata['feat'])[val_mask], g.ndata['label'][val_mask]).item()
    print(f"Epoch {epoch:05d} | Loss {loss:.4f} | Train Accuracy {train_acc:.4f} | Val Accuracy {val_acc:.4f} | Val Loss {val_loss:.4f}")

    # Early Stopping 체크
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 00000 | Loss 0.6937 | Train Accuracy 0.5257 | Val Accuracy 0.4943 | Val Loss 0.6932
Epoch 00001 | Loss 0.6930 | Train Accuracy 0.5328 | Val Accuracy 0.5047 | Val Loss 0.6931
Epoch 00002 | Loss 0.6934 | Train Accuracy 0.5458 | Val Accuracy 0.5038 | Val Loss 0.6931
Epoch 00003 | Loss 0.6932 | Train Accuracy 0.5525 | Val Accuracy 0.5085 | Val Loss 0.6930
Epoch 00004 | Loss 0.6920 | Train Accuracy 0.5525 | Val Accuracy 0.5104 | Val Loss 0.6929
Epoch 00005 | Loss 0.6925 | Train Accuracy 0.5470 | Val Accuracy 0.5152 | Val Loss 0.6929
Epoch 00006 | Loss 0.6922 | Train Accuracy 0.5477 | Val Accuracy 0.5114 | Val Loss 0.6929
Epoch 00007 | Loss 0.6931 | Train Accuracy 0.5392 | Val Accuracy 0.5009 | Val Loss 0.6928
Epoch 00008 | Loss 0.6923 | Train Accuracy 0.5371 | Val Accuracy 0.5028 | Val Loss 0.6927
Epoch 00009 | Loss 0.6924 | Train Accuracy 0.5368 | Val Accuracy 0.5019 | Val Loss 0.6927
Epoch 00010 | Loss 0.6921 | Train Accuracy 0.5364 | Val Accuracy 0.5047 | Val Loss 0.6926
Epoch 0001

In [11]:
import pandas as pd
import torch

# test 데이터 로드
test = pd.read_csv('test.csv')

# Claim 텍스트 및 키워드 추출
test_claims = test['claim'].tolist()
test_keywords = test['keybert_keywords'].apply(eval).tolist()

# Claim을 이용한 feature 생성 (학습 데이터와 동일한 방식)
test_tfidf_matrix = tfidf_vectorizer.transform(test_claims).toarray()
test_features = torch.tensor(test_tfidf_matrix, dtype=torch.float)

# 그래프에 feature 추가
test_g = dgl.graph([], num_nodes=len(test))
test_g = dgl.add_self_loop(test_g)
test_g.ndata['feat'] = test_features

# 모델을 평가 모드로 전환
model.eval()

# 테스트 데이터에 대한 예측 수행
with torch.no_grad():
    test_logits = model(test_g, test_g.ndata['feat'])
    _, test_pred = torch.max(test_logits, dim=1)

# 예측 결과를 DataFrame으로 변환
submission = pd.DataFrame({
    'ID': test['id'],
    'Label': test_pred.numpy()
})

# 예측 결과를 submission.csv 파일로 저장
submission.to_csv('submission_128_l++.csv', index=False)

print("Predictions saved to submission.csv")

Predictions saved to submission.csv
